# 🚀 Hermes - Backtesting Chunked de Stratégies

## Architecture
Ce notebook suit l'architecture Medallion de Hermes :
- **Source** : Hub Features Gold (indicateurs pré-calculés)
- **Traitement** : Chunks avec continuité pour gros volumes
- **Analyse** : VectorBT pour validation des stratégies
- **Sortie** : Table test pour résultats intermédiaires

## Workflow
1. **Configuration** : Imports et paramètres
2. **Connexion Sources** : Hub Features Gold (indicateurs pré-calculés)
3. **Stratégie Chunked** : Génération signaux par chunks
4. **Validation VectorBT** : Analyse des performances

**Important** : Tous les indicateurs doivent être pré-calculés dans le Hub Features Gold

---

## 1. 📦 Configuration et Imports

In [1]:
# Imports essentiels
import os
import json
import polars as pl
import numpy as np
import duckdb
import vectorbt as vbt
from pathlib import Path
from typing import Dict, Any, Optional, List, Tuple
from datetime import datetime, timedelta
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

print(f"✅ Imports chargés")
print(f"🐍 Python: {os.sys.version.split()[0]}")
print(f"📊 Polars: {pl.__version__}")
print(f"🧮 VectorBT: {vbt.__version__}")
print(f"🦆 DuckDB: {duckdb.__version__}")

✅ Imports chargés
🐍 Python: 3.10.18
📊 Polars: 0.20.31
🧮 VectorBT: 0.25.5
🦆 DuckDB: 0.9.2


In [ ]:
@dataclass
class BacktestConfig:
    """Configuration pour le backtesting chunked"""
    
    # === PARAMÈTRES DE DONNÉES ===
    provider: str = "binance"
    market: str = "spot"
    data_frequency: str = "monthly"
    data_category: str = "klines"
    symbol: str = "BTCUSDT"
    interval: str = "4h"
    
    # === ARCHITECTURE MEDALLION ===
    gold_bucket: str = "gold"
    test_bucket: str = "test"
    
    # Paramètres de chunking
    chunk_size: int = 50_000  # Lignes par chunk
    overlap_window: int = 100  # Lignes de contexte entre chunks
    
    # Fenêtre temporelle (optionnel - None = tout l'historique)
    start_date: Optional[str] = "2023-01-01"  # Format: "YYYY-MM-DD" ou None
    end_date: Optional[str] = None
    
    # Paramètres de stratégie par défaut
    rsi_oversold: int = 30
    rsi_neutral_low: int = 45
    rsi_neutral_high: int = 55
    ema_fast: int = 12
    ema_slow: int = 26
    supertrend_period: int = 10
    supertrend_multiplier: float = 3.0

    # Buffer contexte - calculé automatiquement
    min_context_buffer: int = 50  # Minimum de sécurité

    def get_required_context_size(self) -> int:
        """Calcule la taille de contexte requise selon les indicateurs"""
        # Prendre le plus grand indicateur + marge de sécurité
        max_indicator_period = max([
            self.ema_fast,
            self.ema_slow, 
            self.supertrend_period,
            14,  # RSI par défaut
            20,  # Bollinger Bands par défaut
            26   # MACD par défaut
        ])

        # Ajouter une marge de sécurité (50% du plus grand indicateur)
        safety_margin = int(max_indicator_period * 0.5)
        required_size = max_indicator_period + safety_margin
        
        # S'assurer d'avoir au moins le minimum
        return max(required_size, self.min_context_buffer, self.overlap_window)
    
    # Backtesting
    initial_cash: float = 10000.0
    fees: float = 0.001  # 0.1%
    
    # MinIO
    minio_endpoint: str = "127.0.0.1:9000"
    minio_access_key: str = "minioadm"
    minio_secret_key: str = "minioadm"
    
    # === SOURCES DE DONNÉES (CHEMINS CALCULÉS) ===
    use_test_features: bool = False  # Ajouter les features test aux features de base
    
    @property
    def feature_store_table(self) -> str:
        """Nom de la table Gold Features principale (base)"""
        return f"gold_features_{self.market}_{self.data_frequency}_{self.data_category}_{self.symbol}_{self.interval}"
    
    @property
    def feature_store_test_table(self) -> str:
        """Nom de la table Gold Features Test (features supplémentaires)"""
        return f"gold_features_test_{self.market}_{self.data_frequency}_{self.data_category}_{self.symbol}_{self.interval}"
    
    @property
    def feature_store_path(self) -> str:
        """Chemin source Features principales (toujours utilisées)"""
        return f"s3://{self.gold_bucket}/{self.feature_store_table}/**/*.parquet"
    
    @property
    def feature_store_test_path(self) -> str:
        """Chemin source Features Test (colonnes supplémentaires)"""
        return f"s3://{self.test_bucket}/{self.feature_store_test_table}/**/*.parquet"
    
    @property
    def bronze_path(self) -> str:
        """Chemin Bronze pour référence"""
        return f"s3://bronze/{self.provider}/data/{self.market}/{self.data_frequency}/{self.data_category}/{self.symbol}/{self.interval}/**/*.parquet"
    
    # Sortie
    test_table_path: str = "s3://test/backtest_results/"
    
    def get_indicator_columns(self) -> Dict[str, str]:
        """Mapping des colonnes d'indicateurs"""
        return {
            "ema_fast": f"ema_{self.ema_fast}",
            "ema_slow": f"ema_{self.ema_slow}",
            "rsi_14": "rsi_14",
            "supertrend": f"supertrend_{self.supertrend_period}_{self.supertrend_multiplier}",
            "supertrend_dir": f"supertrend_dir_{self.supertrend_period}_{self.supertrend_multiplier}",
            "bb_upper": "bb_upper_20_2",
            "bb_middle": "bb_middle_20_2", 
            "bb_lower": "bb_lower_20_2",
            "macd": "macd_12_26_9",
            "macd_signal": "macd_signal_12_26_9",
            "atr_14": "atr_14"
        }

# Configuration par défaut
config = BacktestConfig()

print("⚙️ Configuration initialisée")
print(f"📊 Provider: {config.provider} | Symbole: {config.symbol} | Intervalle: {config.interval}")
print(f"🔄 Chunk size: {config.chunk_size:,} lignes")
print(f"🛡️ Buffer contexte: {config.get_required_context_size()} lignes")
print(f"📅 Période: {config.start_date} → {config.end_date or 'fin'}")
print(f"💰 Capital initial: ${config.initial_cash:,.2f}")

print(f"\n📁 CHEMINS ARCHITECTURE HERMES:")
print(f"   • Features Base (toujours): {config.feature_store_path}")
print(f"   • Features Test (ajout): {config.feature_store_test_path if config.use_test_features else 'Non utilisées'}")
print(f"   • Table Base: {config.feature_store_table}")
print(f"   • Table Test: {config.feature_store_test_table}")
print(f"   • Bronze (référence): {config.bronze_path}")

print(f"\n🎯 ARCHITECTURE DE CONCATÉNATION:")
print(f"   • Base Features: {config.feature_store_table} (OHLCV + indicateurs validés)")
print(f"   • Test Features: {config.feature_store_test_table} (colonnes supplémentaires)")
print(f"   • Mode actuel: {'Base + Test (concaténation)' if config.use_test_features else 'Base uniquement'}")
print(f"   💡 Changez use_test_features=True pour ajouter les features expérimentales")

⚙️ Configuration initialisée
📊 Symbole: BTCUSDT
🔄 Chunk size: 50,000 lignes
🛡️ Buffer contexte: 100 lignes
📅 Période: 2023-01-01 → fin
💰 Capital initial: $10,000.00


### 🏛️ Architecture de Concaténation de Features

**Chemins selon Architecture Hermes :**

```python
# Features Base (toujours chargées)
s3://gold/gold_features_{market}_{frequency}_{category}_{symbol}_{interval}/**/*.parquet
↓
s3://gold/gold_features_spot_monthly_klines_BTCUSDT_4h/**/*.parquet
(OHLCV + indicateurs validés : EMA, RSI, SuperTrend, etc.)

# Features Test (ajoutées si use_test_features=True)
s3://test/gold_features_test_{market}_{frequency}_{category}_{symbol}_{interval}/**/*.parquet
↓  
s3://test/gold_features_test_spot_monthly_klines_BTCUSDT_4h/**/*.parquet
(Colonnes supplémentaires : nouveaux indicateurs expérimentaux)

# Bronze (référence)
s3://bronze/{provider}/data/{market}/{frequency}/{category}/{symbol}/{interval}/**/*.parquet
```

**Mode de Concaténation :**
- `use_test_features=False` → Features base uniquement
- `use_test_features=True` → Features base + JOIN avec features test (par datetime)

**Avantages :**
- ✅ **Efficacité** : Pas de recalcul des features de base
- ✅ **Modularité** : Ajout incrémental de nouvelles features
- ✅ **Performance** : Jointure optimisée par DuckDB
- ✅ **Flexibilité** : Test isolé des nouvelles features sans impact sur la base

---

In [ ]:
# 🔄 CONFIGURATION ALTERNATIVE (optionnel)
# Décommentez pour tester avec d'autres paramètres ou sources

# config_test = BacktestConfig(
#     symbol="ETHUSDT",           # Autre crypto
#     interval="1h",              # Autre intervalle  
#     data_frequency="daily",     # Autre fréquence
#     use_test_features=True,     # Ajouter features expérimentales
#     start_date="2024-01-01"     # Période plus courte
# )
# 
# print("🔄 Configuration alternative:")
# print(f"   • Features Base: {config_test.feature_store_path}")
# print(f"   • Features Test: {config_test.feature_store_test_path}")
# print(f"   • Mode: {'Base + Test' if config_test.use_test_features else 'Base uniquement'}")

print("💡 Pour tester avec features supplémentaires:")
print("   1. Créez d'abord la table gold_features_test avec vos nouvelles colonnes")
print("   2. Changez config.use_test_features = True") 
print("   3. Le système chargera automatiquement Base + Test en jointure")
print("   4. Ou utilisez une config alternative comme ci-dessus")

## 2. 🔌 Connexion aux Sources de Données

In [ ]:
class HermesDataLoader:
    """Gestionnaire de connexion aux données Hermes avec support concaténation features"""
    
    def __init__(self, config: BacktestConfig):
        self.config = config
        self.con = None
        self.indicator_cols = config.get_indicator_columns()
        
        # Colonnes de base OHLCV
        self.price_cols = ['datetime', 'open', 'high', 'low', 'close', 'volume']
        
        # Validation de la disponibilité des tables
        self.base_table_available = False
        self.test_table_available = False
        
    def setup_connection(self):
        """Configure la connexion DuckDB vers MinIO"""
        print("🔌 Configuration connexion DuckDB → MinIO...")
        
        self.con = duckdb.connect()
        
        # Configuration S3/MinIO
        self.con.execute(f"""
            SET s3_access_key_id='{self.config.minio_access_key}';
            SET s3_secret_access_key='{self.config.minio_secret_key}';
            SET s3_endpoint='{self.config.minio_endpoint}';
            SET s3_url_style='path';
            SET s3_use_ssl='false';
        """)
        
        # Optimisations mémoire
        self.con.execute("""
            SET threads TO 6;
            SET memory_limit = '4GB';
            SET enable_progress_bar = true;
        """)
        
        print("✅ Connexion DuckDB configurée")
    
    def get_partition_info(self) -> List[Dict]:
        """Récupère les informations des partitions disponibles"""
        if not self.con:
            self.setup_connection()
        
        print("📊 Analyse des partitions disponibles...")
        
        try:
            # Récupérer la liste des fichiers avec métadonnées
            result = self.con.execute(f"""
                SELECT file as filename
                FROM glob('{self.config.feature_store_path}')
                ORDER BY file
            """).fetchall()
            
            partitions = []
            for row in result:
                partitions.append({
                    'path': row[0],
                    'size_mb': 0,  # Taille non disponible avec glob simple
                    'last_modified': 'unknown'
                })
            
            print(f"📁 {len(partitions)} partitions trouvées")
            if partitions:
                print(f"📁 Fichiers trouvés:")
                for i, p in enumerate(partitions[:5]):  # Afficher les 5 premiers
                    print(f"   • {p['path']}")
                if len(partitions) > 5:
                    print(f"   • ... et {len(partitions)-5} autres")
            
            return partitions
            
        except Exception as e:
            print(f"❌ Erreur lors de l'analyse des partitions: {e}")
            return []
    
    def validate_tables_availability(self) -> Dict[str, bool]:
        """Valide la disponibilité des tables base et test"""
        if not self.con:
            self.setup_connection()
        
        print("🔍 Validation des tables disponibles...")
        
        # Test table base
        try:
            base_result = self.con.execute(f"""
                SELECT COUNT(*) FROM read_parquet('{self.config.feature_store_path}') LIMIT 1
            """).fetchone()
            self.base_table_available = True
            print(f"   ✅ Table base disponible: {self.config.feature_store_table}")
        except Exception as e:
            self.base_table_available = False
            print(f"   ❌ Table base indisponible: {e}")
        
        # Test table test si demandée
        if self.config.use_test_features:
            try:
                test_result = self.con.execute(f"""
                    SELECT COUNT(*) FROM read_parquet('{self.config.feature_store_test_path}') LIMIT 1
                """).fetchone()
                self.test_table_available = True
                print(f"   ✅ Table test disponible: {self.config.feature_store_test_table}")
            except Exception as e:
                self.test_table_available = False
                print(f"   ⚠️ Table test indisponible: {e}")
        else:
            print(f"   ℹ️ Table test non demandée (use_test_features=False)")
        
        return {
            'base_available': self.base_table_available,
            'test_available': self.test_table_available
        }
    
    def get_combined_query(self, where_clause: str = "", limit_clause: str = "") -> str:
        """Génère la requête combinée base + test si disponible"""
        
        base_query = f"SELECT * FROM read_parquet('{self.config.feature_store_path}')"
        
        if self.config.use_test_features and self.test_table_available:
            # Requête avec JOIN entre base et test
            combined_query = f"""
                SELECT 
                    base.*,
                    test.* EXCLUDE (datetime, symbol, open, high, low, close, volume)
                FROM ({base_query}) base
                LEFT JOIN (
                    SELECT * FROM read_parquet('{self.config.feature_store_test_path}')
                ) test ON base.datetime = test.datetime AND base.symbol = test.symbol
            """
            print(f"🔗 Mode: Base + Test (JOIN par datetime)")
        else:
            combined_query = base_query
            print(f"📊 Mode: Base uniquement")
        
        # Ajouter les clauses WHERE et LIMIT
        if where_clause:
            combined_query = f"SELECT * FROM ({combined_query}) main {where_clause}"
        
        if limit_clause:
            combined_query += f" {limit_clause}"
        
        return combined_query
    
    def get_data_summary(self) -> Dict:
        """Récupère un résumé des données disponibles (combinées si applicable)"""
        if not self.con:
            self.setup_connection()
        
        # Valider les tables d'abord
        self.validate_tables_availability()
        
        if not self.base_table_available:
            print("❌ Table base indisponible - impossible de continuer")
            return {}
        
        print("📈 Analyse du contenu des données...")
        
        # Requête avec filtre temporel si spécifié
        where_clause = f"WHERE symbol = '{self.config.symbol}'"
        if self.config.start_date:
            where_clause += f" AND datetime >= '{self.config.start_date}'"
        if self.config.end_date:
            where_clause += f" AND datetime <= '{self.config.end_date}'"
        
        try:
            # Utiliser la requête combinée
            combined_query = self.get_combined_query(where_clause)
            
            result = self.con.execute(f"""
                SELECT 
                    COUNT(*) as total_rows,
                    MIN(datetime) as start_date,
                    MAX(datetime) as end_date,
                    COUNT(DISTINCT date_trunc('day', datetime)) as unique_days
                FROM ({combined_query}) data
            """).fetchone()
            
            summary = {
                'total_rows': result[0],
                'start_date': result[1],
                'end_date': result[2],
                'unique_days': result[3],
                'using_test_features': self.config.use_test_features and self.test_table_available
            }
            
            print(f"📊 Résumé des données {'(Base + Test)' if summary['using_test_features'] else '(Base)'}:")
            print(f"   • Total lignes: {summary['total_rows']:,}")
            print(f"   • Période: {summary['start_date']} → {summary['end_date']}")
            print(f"   • Jours uniques: {summary['unique_days']:,}")
            
            # Estimation des chunks
            estimated_chunks = (summary['total_rows'] // self.config.chunk_size) + 1
            print(f"   • Chunks estimés: {estimated_chunks:,}")
            
            return summary
            
        except Exception as e:
            print(f"❌ Erreur lors de l'analyse: {e}")
            return {}

# Initialisation du loader
data_loader = HermesDataLoader(config)

# 🔍 Test de connexion et validation des chemins (base + test)
print("🔍 TEST DE CONNEXION ET VALIDATION DES CHEMINS")
print("=" * 50)

# Test direct des chemins avec glob
try:
    if not data_loader.con:
        data_loader.setup_connection()
    
    # Test table BASE (obligatoire)
    print(f"\n📊 TEST TABLE BASE:")
    base_test_paths = [
        "s3://gold/gold_features_spot_monthly_klines_BTCUSDT_4h/**/*.parquet",  # Pattern actuel
        "s3://gold/gold_features_spot_monthly_klines_BTCUSDT_4h/*.parquet",     # Pattern direct
        "s3://gold/gold_features_spot_monthly_klines_BTCUSDT_4h/*",             # Tous les fichiers
    ]
    
    base_path_found = False
    for i, test_path in enumerate(base_test_paths, 1):
        print(f"🧪 Test {i}: {test_path}")
        try:
            # Test avec glob pour lister les fichiers
            files_result = data_loader.con.execute(f"""
                SELECT file as filename
                FROM glob('{test_path}')
                LIMIT 5
            """).fetchall()
            
            if files_result:
                print(f"   ✅ {len(files_result)} fichier(s) base trouvé(s)")
                
                # Si on trouve des fichiers, tester la lecture
                try:
                    test_read = data_loader.con.execute(f"""
                        SELECT COUNT(*) as row_count, COUNT(DISTINCT symbol) as symbols
                        FROM read_parquet('{test_path}')
                        LIMIT 1
                    """).fetchone()
                    
                    if test_read:
                        print(f"   📊 Test lecture base: {test_read[0]:,} lignes, {test_read[1]} symbole(s)")
                        # Mettre à jour la config avec le chemin qui fonctionne
                        # Extraire le pattern et l'appliquer à la propriété
                        config.feature_store_path = test_path
                        base_path_found = True
                        print(f"   🎯 Chemin base optimal trouvé !")
                        break
                        
                except Exception as read_error:
                    print(f"   ❌ Erreur lecture base: {read_error}")
            else:
                print(f"   ❌ Aucun fichier base trouvé")
                
        except Exception as e:
            print(f"   ❌ Erreur test base: {e}")
    
    # Test table TEST (si demandée)
    if config.use_test_features:
        print(f"\n🧪 TEST TABLE TEST:")
        test_table_paths = [
            "s3://test/gold_features_test_spot_monthly_klines_BTCUSDT_4h/**/*.parquet",
            "s3://test/gold_features_test_spot_monthly_klines_BTCUSDT_4h/*.parquet",
            "s3://test/gold_features_test_spot_monthly_klines_BTCUSDT_4h/*",
        ]
        
        test_path_found = False
        for i, test_path in enumerate(test_table_paths, 1):
            print(f"🧪 Test {i}: {test_path}")
            try:
                files_result = data_loader.con.execute(f"""
                    SELECT file as filename
                    FROM glob('{test_path}')
                    LIMIT 3
                """).fetchall()
                
                if files_result:
                    print(f"   ✅ {len(files_result)} fichier(s) test trouvé(s)")
                    try:
                        test_read = data_loader.con.execute(f"""
                            SELECT COUNT(*) as row_count
                            FROM read_parquet('{test_path}')
                            LIMIT 1
                        """).fetchone()
                        
                        if test_read:
                            print(f"   📊 Test lecture test: {test_read[0]:,} lignes")
                            test_path_found = True
                            print(f"   🎯 Chemin test trouvé !")
                            break
                            
                    except Exception as read_error:
                        print(f"   ❌ Erreur lecture test: {read_error}")
                else:
                    print(f"   ❌ Aucun fichier test trouvé")
                    
            except Exception as e:
                print(f"   ❌ Erreur test: {e}")
        
        if not test_path_found:
            print(f"   ⚠️ Table test non trouvée - passage en mode base uniquement")
            config.use_test_features = False
    else:
        print(f"\n💡 Table test non demandée (use_test_features=False)")
    
    print(f"\n🎯 CONFIGURATION FINALE:")
    print(f"   • Chemin base: {config.feature_store_path}")
    if config.use_test_features:
        print(f"   • Chemin test: {config.feature_store_test_path}")
        print(f"   • Mode: Base + Test (concaténation)")
    else:
        print(f"   • Mode: Base uniquement")
    
except Exception as e:
    print(f"❌ Erreur lors du test de connexion: {e}")

print("\n" + "=" * 50)

# Maintenant charger les infos avec le bon chemin
partitions = data_loader.get_partition_info()
data_summary = data_loader.get_data_summary()

🔍 TEST DE CONNEXION ET VALIDATION DU CHEMIN
🔌 Configuration connexion DuckDB → MinIO...
✅ Connexion DuckDB configurée

🧪 Test 1: s3://gold/gold_features_spot_monthly_klines_BTCUSDT_4h/**/*.parquet
   ✅ 5 fichier(s) trouvé(s)
     📁 s3://gold/gold_features_spot_monthly_klines_BTCUSDT_4h/year=2017/month=10/data_0.parquet
     📁 s3://gold/gold_features_spot_monthly_klines_BTCUSDT_4h/year=2017/month=11/data_0.parquet
     📁 s3://gold/gold_features_spot_monthly_klines_BTCUSDT_4h/year=2017/month=12/data_0.parquet
     ... et 2 autres
   📊 Test lecture: 17,604 lignes, 1 symbole(s)
   🎯 Chemin optimal trouvé !

🎯 Chemin final utilisé: s3://gold/gold_features_spot_monthly_klines_BTCUSDT_4h/**/*.parquet

📊 Analyse des partitions disponibles...
📁 97 partitions trouvées
📁 Fichiers trouvés:
   • s3://gold/gold_features_spot_monthly_klines_BTCUSDT_4h/year=2017/month=10/data_0.parquet
   • s3://gold/gold_features_spot_monthly_klines_BTCUSDT_4h/year=2017/month=11/data_0.parquet
   • s3://gold/gold_fea

## 3. 📊 Processeur de Backtesting Chunked avec Garanties de Cohérence

### Système de traitement par chunks avec continuité des signaux et positions

**🛡️ Garanties de Cohérence Implémentées** :

1. **Buffer de Contexte Étendu** : 50+ lignes de contexte entre chunks
2. **Validation des Signaux `shift()`** : Vérification que les valeurs précédentes existent
3. **Détection de Chevauchements** : Gestion automatique des doublons temporels
4. **Validation Continue** : Contrôles à chaque étape du traitement
5. **Diagnostic Préalable** : Vérification de la cohérence des données source

**Note** : Ce processeur utilise uniquement les indicateurs pré-calculés dans le Hub Features Gold

In [ ]:
class HermesChunkedBacktester:
    """Backtesting chunked avec continuité pour stratégies Hermes"""
    
    def __init__(self, config: BacktestConfig, data_loader: HermesDataLoader):
        self.config = config
        self.data_loader = data_loader
        self.indicator_cols = config.get_indicator_columns()
        
        # État persistant entre chunks
        self.state = {
            'context_rows': None,        # Lignes de contexte pour continuité
            'last_position': None,       # 'long', 'short' ou None
            'cumulative_cash': config.initial_cash,
            'cumulative_value': config.initial_cash,
            'total_trades': 0,
            'chunk_results': [],         # Résultats par chunk
            'chunk_counter': 0,
            'context_buffer_size': config.get_required_context_size()  #  Calcul automatique du buffer plus grand pour les signaux
        }
    
    def compute_strategy_signals(self, df: pl.DataFrame) -> pl.DataFrame:
        """Calcule les signaux de la stratégie Smart Momentum avec validation de cohérence"""
        
        # Récupérer les colonnes d'indicateurs
        ema_fast_col = self.indicator_cols["ema_fast"]
        ema_slow_col = self.indicator_cols["ema_slow"]
        rsi_col = self.indicator_cols["rsi_14"]
        supertrend_dir_col = self.indicator_cols["supertrend_dir"]
        
        # Vérifier que les colonnes existent
        missing_cols = []
        for col_name, col_actual in self.indicator_cols.items():
            if col_actual not in df.columns:
                missing_cols.append(f"{col_name} -> {col_actual}")
        
        if missing_cols:
            raise ValueError(f"❌ Colonnes d'indicateurs manquantes: {missing_cols}")
        
        # Calcul des signaux avec validation de continuité
        signals_df = df.with_columns([
            # === VALIDATION DE CONTINUITÉ ===
            # Marquer les lignes où shift(1) sera valide
            (pl.int_range(pl.len()) > 0).alias("has_previous_value"),
            
            # === CONDITIONS EMA ===
            # Condition actuelle : EMA rapide > EMA lente
            (pl.col(ema_fast_col) > pl.col(ema_slow_col)).alias("ema_fast_above_slow"),
            
            # Condition précédente : EMA rapide <= EMA lente (avec gestion des nulls)
            (pl.col(ema_fast_col).shift(1) <= pl.col(ema_slow_col).shift(1)).alias("ema_was_below_or_equal"),
            
            # Crossover EMA seulement si on a une valeur précédente valide
            ((pl.col(ema_fast_col) > pl.col(ema_slow_col)) & 
             (pl.col(ema_fast_col).shift(1) <= pl.col(ema_slow_col).shift(1)) &
             (pl.int_range(pl.len()) > 0)).alias("ema_bullish_cross"),
            
            # === CONDITIONS RSI ===
            ((pl.col(rsi_col) >= self.config.rsi_neutral_low) & 
             (pl.col(rsi_col) <= self.config.rsi_neutral_high)).alias("rsi_neutral"),
            
            # === CONDITIONS SUPERTREND ===
            (pl.col(supertrend_dir_col) == 1).alias("supertrend_bullish"),
            
            # SuperTrend exit avec validation de continuité
            ((pl.col(supertrend_dir_col).shift(1) == 1) & 
             (pl.col(supertrend_dir_col) == -1) &
             (pl.int_range(pl.len()) > 0)).alias("supertrend_exit")
        ])
        
        # Signaux finaux avec validation
        final_signals = signals_df.with_columns([
            # Signal d'entrée (achat) - uniquement si toutes conditions remplies
            (pl.col("ema_bullish_cross") & 
             pl.col("rsi_neutral") & 
             pl.col("supertrend_bullish")).alias("buy_signal"),
            
            # Signal de sortie (vente) - uniquement avec continuité validée
            (pl.col("supertrend_exit")).alias("sell_signal")
        ])
        
        return final_signals
    
    def load_chunk_with_context(self, offset: int, limit: int) -> pl.DataFrame:
        """Charge un chunk avec le contexte nécessaire pour garantir la cohérence (base + test)"""
        
        # Construire la requête avec filtre temporel
        where_clause = f"WHERE symbol = '{self.config.symbol}'"
        if self.config.start_date:
            where_clause += f" AND datetime >= '{self.config.start_date}'"
        if self.config.end_date:
            where_clause += f" AND datetime <= '{self.config.end_date}'"
        
        # AMÉLIORATION : Charger plus de contexte pour les premiers chunks
        actual_offset = offset
        actual_limit = limit
        
        # Si ce n'est pas le premier chunk, commencer plus tôt pour avoir du contexte
        if offset > 0 and self.state['context_rows'] is None:
            # Charger du contexte supplémentaire depuis la base
            context_needed = self.state['context_buffer_size']
            actual_offset = max(0, offset - context_needed)
            actual_limit = limit + (offset - actual_offset)
        
        # Utiliser la requête combinée (base + test si disponible)
        base_query = self.data_loader.get_combined_query(where_clause)
        
        # Requête DuckDB pour le chunk avec contexte
        query = f"""
            SELECT * FROM ({base_query}) data
            ORDER BY datetime
            LIMIT {actual_limit}
            OFFSET {actual_offset}
        """
        
        # Exécuter la requête
        result = self.data_loader.con.execute(query).arrow()
        chunk_df = pl.from_arrow(result)
        
        # Ajouter le contexte des lignes précédentes si disponible
        if self.state['context_rows'] is not None and offset > 0:
            # Vérifier la continuité temporelle
            if len(self.state['context_rows']) > 0 and len(chunk_df) > 0:
                last_context_time = self.state['context_rows']['datetime'].max()
                first_chunk_time = chunk_df['datetime'].min()
                
                if last_context_time >= first_chunk_time:
                    print(f"⚠️ Chevauchement temporel détecté - ajustement automatique")
                    # Filtrer les doublons
                    chunk_df = chunk_df.filter(pl.col('datetime') > last_context_time)
            
            # Concaténer avec le contexte
            if len(chunk_df) > 0:
                chunk_df = pl.concat([self.state['context_rows'], chunk_df])
        
        return chunk_df
    
    def process_chunk(self, chunk_df: pl.DataFrame, is_first_chunk: bool) -> Dict:
        """Traite un chunk avec calcul des signaux et backtesting"""
        
        if len(chunk_df) == 0:
            return {
                'chunk_id': self.state['chunk_counter'],
                'rows_processed': 0,
                'buy_signals': 0,
                'sell_signals': 0,
                'start_time': None,
                'end_time': None,
                'data': None,
                'warnings': ['Chunk vide']
            }
        
        # 1. Calculer les signaux de stratégie avec validation de cohérence
        try:
            signals_df = self.compute_strategy_signals(chunk_df)
        except ValueError as e:
            print(f"❌ Erreur dans le calcul des signaux: {e}")
            return {
                'chunk_id': self.state['chunk_counter'],
                'rows_processed': 0,
                'buy_signals': 0,
                'sell_signals': 0,
                'error': str(e)
            }
        
        # 2. Ajuster les signaux selon l'état précédent
        if not is_first_chunk and self.state['last_position'] is not None:
            # Si on était en position, ne pas générer d'entrée immédiate
            if self.state['last_position'] == 'long':
                signals_df = signals_df.with_columns(
                    pl.when(pl.int_range(pl.len()) == 0)
                    .then(False)
                    .otherwise(pl.col("buy_signal"))
                    .alias("buy_signal")
                )
        
        # 3. Extraire les résultats sans le contexte
        context_size = len(self.state['context_rows']) if self.state['context_rows'] is not None and not is_first_chunk else 0
        
        if context_size > 0:
            result_df = signals_df.slice(context_size)
        else:
            result_df = signals_df
        
        # 4. Validation des signaux calculés
        warnings = []
        if len(result_df) > 0:
            # Vérifier qu'on n'a pas de signaux sur la première ligne d'un chunk (sauf premier chunk)
            if not is_first_chunk and context_size == 0:
                first_row_signals = result_df.head(1)
                if (first_row_signals.select(pl.col("buy_signal").sum()).item() > 0 or 
                    first_row_signals.select(pl.col("sell_signal").sum()).item() > 0):
                    warnings.append("Signaux détectés sur première ligne sans contexte")
        
        # 5. Compter les signaux
        buy_signals = result_df.select(pl.col("buy_signal").sum()).item() if len(result_df) > 0 else 0
        sell_signals = result_df.select(pl.col("sell_signal").sum()).item() if len(result_df) > 0 else 0
        
        # 6. Mettre à jour l'état pour le chunk suivant avec plus de contexte
        if len(signals_df) > 0:
            self.state['context_rows'] = signals_df.tail(self.state['context_buffer_size'])
        
        # Simuler la position (logique simplifiée)
        if buy_signals > 0 and self.state['last_position'] != 'long':
            self.state['last_position'] = 'long'
        elif sell_signals > 0 and self.state['last_position'] == 'long':
            self.state['last_position'] = None
            self.state['total_trades'] += 1
        
        # 7. Retourner les résultats du chunk avec métadonnées de validation
        chunk_result = {
            'chunk_id': self.state['chunk_counter'],
            'rows_processed': len(result_df),
            'buy_signals': buy_signals,
            'sell_signals': sell_signals,
            'start_time': result_df.select(pl.col("datetime").min()).item() if len(result_df) > 0 else None,
            'end_time': result_df.select(pl.col("datetime").max()).item() if len(result_df) > 0 else None,
            'data': result_df,  # Conserver les données pour sauvegarde
            'context_size': context_size,
            'warnings': warnings,
            'continuity_validated': context_size > 0 or is_first_chunk
        }
        
        self.state['chunk_counter'] += 1
        
        return chunk_result

# Initialisation du backtester
backtester = HermesChunkedBacktester(config, data_loader)

print("🚀 Backtester chunked initialisé avec garanties de cohérence")
print(f"⚙️ Configuration: {config.chunk_size:,} lignes/chunk avec {backtester.state['context_buffer_size']} lignes de contexte")
print(f"💰 Capital initial: ${config.initial_cash:,.2f}")
print("📊 Features de base toujours chargées depuis la table Gold")
if config.use_test_features:
    print("🧪 Features test ajoutées par JOIN (nouvelles colonnes expérimentales)")
else:
    print("🏭 Mode production: features de base uniquement")
print("✅ Système de validation de continuité des signaux activé")

🚀 Backtester chunked initialisé avec garanties de cohérence
⚙️ Configuration: 50,000 lignes/chunk avec 100 lignes de contexte
💰 Capital initial: $10,000.00
📊 Tous les indicateurs doivent être pré-calculés dans le Hub Features Gold
✅ Système de validation de continuité des signaux activé


## 4. 🔄 Exécution du Backtesting Chunked

## 4.1 🔍 Diagnostic de Cohérence des Données

Avant d'exécuter le backtesting, vérifions la cohérence des données et la continuité temporelle.

In [5]:
def run_coherence_diagnostic() -> Dict:
    """Diagnostic de cohérence des données avant backtesting"""
    
    print("🔍 DIAGNOSTIC DE COHERENCE DES DONNÉES")
    print("=" * 40)
    
    if not data_loader.con:
        data_loader.setup_connection()
    
    diagnostic_results = {
        'temporal_continuity': True,
        'indicator_completeness': True,
        'data_gaps': [],
        'missing_indicators': [],
        'recommendations': []
    }
    
    try:
        # 1. Vérifier la continuité temporelle
        print("📅 Vérification de la continuité temporelle...")
        
        temporal_query = f"""
            WITH time_diffs AS (
                SELECT 
                    datetime,
                    LAG(datetime) OVER (ORDER BY datetime) as prev_datetime,
                    datetime - LAG(datetime) OVER (ORDER BY datetime) as time_diff_interval,
                    EXTRACT(EPOCH FROM (datetime - LAG(datetime) OVER (ORDER BY datetime))) / 3600.0 as time_diff_hours
                FROM read_parquet('{config.feature_store_path}')
                WHERE symbol = '{config.symbol}'
                ORDER BY datetime
                LIMIT 10000  -- Échantillon pour diagnostic
            )
            SELECT 
                COUNT(*) as total_rows,
                COUNT(DISTINCT time_diff_hours) as unique_intervals,
                MIN(time_diff_hours) as min_interval_hours,
                MAX(time_diff_hours) as max_interval_hours,
                AVG(time_diff_hours) as avg_interval_hours
            FROM time_diffs 
            WHERE time_diff_hours IS NOT NULL
        """
        
        result = data_loader.con.execute(temporal_query).fetchone()
        total_rows, unique_intervals, min_interval_hours, max_interval_hours, avg_interval_hours = result
        
        print(f"   • Lignes analysées: {total_rows:,}")
        print(f"   • Intervalles uniques: {unique_intervals}")
        print(f"   • Intervalle min: {min_interval_hours:.2f} heures")
        print(f"   • Intervalle max: {max_interval_hours:.2f} heures")
        print(f"   • Intervalle moyen: {avg_interval_hours:.2f} heures")
        
        if unique_intervals > 2:  # Tolérance pour quelques variations
            diagnostic_results['temporal_continuity'] = False
            diagnostic_results['recommendations'].append(
                "⚠️ Intervalles temporels irréguliers détectés - vérifier la qualité des données"
            )
        
        # 2. Vérifier la présence des indicateurs requis
        print("\n📊 Vérification des indicateurs requis...")
        
        schema_query = f"""
            DESCRIBE SELECT * FROM read_parquet('{config.feature_store_path}') LIMIT 1
        """
        
        available_columns = [row[0] for row in data_loader.con.execute(schema_query).fetchall()]
        required_indicators = list(config.get_indicator_columns().values())
        
        missing_indicators = [ind for ind in required_indicators if ind not in available_columns]
        
        print(f"   • Colonnes disponibles: {len(available_columns)}")
        print(f"   • Indicateurs requis: {len(required_indicators)}")
        print(f"   • Indicateurs manquants: {len(missing_indicators)}")
        
        if missing_indicators:
            diagnostic_results['indicator_completeness'] = False
            diagnostic_results['missing_indicators'] = missing_indicators
            print(f"   ❌ Indicateurs manquants: {missing_indicators}")
            diagnostic_results['recommendations'].append(
                f"❌ Recalculer ou ajouter les indicateurs manquants: {missing_indicators}"
            )
        else:
            print("   ✅ Tous les indicateurs requis sont présents")
        
        # 3. Tester un échantillon de calcul de signaux
        print("\n🧪 Test de calcul de signaux sur échantillon...")
        
        sample_query = f"""
            SELECT *
            FROM read_parquet('{config.feature_store_path}')
            WHERE symbol = '{config.symbol}'
            ORDER BY datetime
            LIMIT 1000
        """
        
        sample_result = data_loader.con.execute(sample_query).arrow()
        sample_df = pl.from_arrow(sample_result)
        
        if len(sample_df) > 0:
            try:
                # Test du calcul de signaux
                test_signals = backtester.compute_strategy_signals(sample_df)
                
                buy_count = test_signals.select(pl.col("buy_signal").sum()).item()
                sell_count = test_signals.select(pl.col("sell_signal").sum()).item()
                
                print(f"   ✅ Test réussi - {buy_count} signaux d'achat, {sell_count} signaux de vente sur échantillon")
                
                # Vérifier les signaux sur la première ligne (problème de shift)
                first_row_signals = test_signals.head(1)
                first_buy = first_row_signals.select(pl.col("buy_signal")).item()
                first_sell = first_row_signals.select(pl.col("sell_signal")).item()
                
                if first_buy or first_sell:
                    diagnostic_results['recommendations'].append(
                        "⚠️ Signaux détectés sur première ligne - vérifier la logique de shift()"
                    )
                
            except Exception as e:
                print(f"   ❌ Erreur dans le calcul de signaux: {e}")
                diagnostic_results['recommendations'].append(f"❌ Erreur calcul signaux: {e}")
        
        # 4. Résumé du diagnostic
        print(f"\n📋 RÉSUMÉ DU DIAGNOSTIC:")
        print(f"   • Continuité temporelle: {'✅' if diagnostic_results['temporal_continuity'] else '❌'}")
        print(f"   • Indicateurs complets: {'✅' if diagnostic_results['indicator_completeness'] else '❌'}")
        
        if diagnostic_results['recommendations']:
            print(f"\n💡 RECOMMANDATIONS:")
            for rec in diagnostic_results['recommendations']:
                print(f"   {rec}")
        else:
            print(f"\n✅ TOUTES LES VÉRIFICATIONS PASSÉES - PRÊT POUR LE BACKTESTING")
        
        return diagnostic_results
        
    except Exception as e:
        print(f"❌ Erreur lors du diagnostic: {e}")
        diagnostic_results['recommendations'].append(f"❌ Erreur diagnostic: {e}")
        return diagnostic_results

# Exécution du diagnostic
diagnostic = run_coherence_diagnostic()

🔍 DIAGNOSTIC DE COHERENCE DES DONNÉES
📅 Vérification de la continuité temporelle...
   • Lignes analysées: 9,999
   • Intervalles uniques: 4
   • Intervalle min: 4.00 heures
   • Intervalle max: 32.00 heures
   • Intervalle moyen: 4.01 heures

📊 Vérification des indicateurs requis...
   • Colonnes disponibles: 35
   • Indicateurs requis: 11
   • Indicateurs manquants: 0
   ✅ Tous les indicateurs requis sont présents

🧪 Test de calcul de signaux sur échantillon...
   ✅ Test réussi - 1 signaux d'achat, 11 signaux de vente sur échantillon

📋 RÉSUMÉ DU DIAGNOSTIC:
   • Continuité temporelle: ❌
   • Indicateurs complets: ✅

💡 RECOMMANDATIONS:
   ⚠️ Intervalles temporels irréguliers détectés - vérifier la qualité des données
   ✅ Test réussi - 1 signaux d'achat, 11 signaux de vente sur échantillon

📋 RÉSUMÉ DU DIAGNOSTIC:
   • Continuité temporelle: ❌
   • Indicateurs complets: ✅

💡 RECOMMANDATIONS:
   ⚠️ Intervalles temporels irréguliers détectés - vérifier la qualité des données


In [6]:
# 🔍 DIAGNOSTIC AVANCÉ : Pourquoi aucun signal n'est généré ?
print("🔍 DIAGNOSTIC AVANCÉ DES SIGNAUX")
print("=" * 35)

# Charger un échantillon plus large pour diagnostic
sample_query = f"""
    SELECT *
    FROM read_parquet('{config.feature_store_path}')
    WHERE symbol = '{config.symbol}'
    ORDER BY datetime
    LIMIT 5000
"""

sample_result = data_loader.con.execute(sample_query).arrow()
sample_df = pl.from_arrow(sample_result)

print(f"📊 Échantillon analysé: {len(sample_df):,} lignes")

if len(sample_df) > 0:
    # Calculer les signaux avec diagnostics détaillés
    ema_fast_col = config.get_indicator_columns()["ema_fast"]
    ema_slow_col = config.get_indicator_columns()["ema_slow"]
    rsi_col = config.get_indicator_columns()["rsi_14"]
    supertrend_dir_col = config.get_indicator_columns()["supertrend_dir"]
    
    # Vérifier les colonnes d'indicateurs
    print(f"\n📈 COLONNES D'INDICATEURS:")
    print(f"   • EMA Fast ({ema_fast_col}): {'✅' if ema_fast_col in sample_df.columns else '❌ MANQUANT'}")
    print(f"   • EMA Slow ({ema_slow_col}): {'✅' if ema_slow_col in sample_df.columns else '❌ MANQUANT'}")
    print(f"   • RSI ({rsi_col}): {'✅' if rsi_col in sample_df.columns else '❌ MANQUANT'}")
    print(f"   • SuperTrend Dir ({supertrend_dir_col}): {'✅' if supertrend_dir_col in sample_df.columns else '❌ MANQUANT'}")
    
    # Analyser les conditions individuelles
    if all(col in sample_df.columns for col in [ema_fast_col, ema_slow_col, rsi_col, supertrend_dir_col]):
        # Calculer les conditions individuelles
        analysis_df = sample_df.with_columns([
            # Conditions EMA
            (pl.col(ema_fast_col) > pl.col(ema_slow_col)).alias("ema_fast_above_slow"),
            (pl.col(ema_fast_col).shift(1) <= pl.col(ema_slow_col).shift(1)).alias("ema_was_below_or_equal"),
            
            # Crossover EMA
            ((pl.col(ema_fast_col) > pl.col(ema_slow_col)) & 
             (pl.col(ema_fast_col).shift(1) <= pl.col(ema_slow_col).shift(1)) &
             (pl.int_range(pl.len()) > 0)).alias("ema_bullish_cross"),
            
            # Conditions RSI
            ((pl.col(rsi_col) >= config.rsi_neutral_low) & 
             (pl.col(rsi_col) <= config.rsi_neutral_high)).alias("rsi_neutral"),
            
            # Conditions SuperTrend
            (pl.col(supertrend_dir_col) == 1).alias("supertrend_bullish"),
            
            # Signal final
            ((pl.col(ema_fast_col) > pl.col(ema_slow_col)) & 
             (pl.col(ema_fast_col).shift(1) <= pl.col(ema_slow_col).shift(1)) &
             (pl.int_range(pl.len()) > 0) &
             (pl.col(rsi_col) >= config.rsi_neutral_low) & 
             (pl.col(rsi_col) <= config.rsi_neutral_high) &
             (pl.col(supertrend_dir_col) == 1)).alias("buy_signal")
        ])
        
        # Statistiques des conditions
        print(f"\n📊 ANALYSE DES CONDITIONS (sur {len(analysis_df):,} lignes):")
        
        ema_fast_above = analysis_df.select(pl.col("ema_fast_above_slow").sum()).item()
        ema_crossover = analysis_df.select(pl.col("ema_bullish_cross").sum()).item()
        rsi_neutral_count = analysis_df.select(pl.col("rsi_neutral").sum()).item()
        supertrend_bull = analysis_df.select(pl.col("supertrend_bullish").sum()).item()
        buy_signals = analysis_df.select(pl.col("buy_signal").sum()).item()
        
        print(f"   • EMA Fast > Slow: {ema_fast_above:,} ({ema_fast_above/len(analysis_df)*100:.1f}%)")
        print(f"   • EMA Bullish Cross: {ema_crossover:,} ({ema_crossover/len(analysis_df)*100:.1f}%)")
        print(f"   • RSI Neutral ({config.rsi_neutral_low}-{config.rsi_neutral_high}): {rsi_neutral_count:,} ({rsi_neutral_count/len(analysis_df)*100:.1f}%)")
        print(f"   • SuperTrend Bullish: {supertrend_bull:,} ({supertrend_bull/len(analysis_df)*100:.1f}%)")
        print(f"   • 🎯 SIGNAUX D'ACHAT FINAUX: {buy_signals:,}")
        
        # Analyser les valeurs RSI pour comprendre le problème
        rsi_stats = sample_df.select([
            pl.col(rsi_col).min().alias("rsi_min"),
            pl.col(rsi_col).max().alias("rsi_max"),
            pl.col(rsi_col).mean().alias("rsi_mean"),
            pl.col(rsi_col).quantile(0.25).alias("rsi_q25"),
            pl.col(rsi_col).quantile(0.75).alias("rsi_q75")
        ]).to_dicts()[0]
        
        print(f"\n📊 STATISTIQUES RSI:")
        print(f"   • Min: {rsi_stats['rsi_min']:.1f}")
        print(f"   • Q25: {rsi_stats['rsi_q25']:.1f}")
        print(f"   • Moyenne: {rsi_stats['rsi_mean']:.1f}")
        print(f"   • Q75: {rsi_stats['rsi_q75']:.1f}")
        print(f"   • Max: {rsi_stats['rsi_max']:.1f}")
        print(f"   • Plage neutre configurée: {config.rsi_neutral_low}-{config.rsi_neutral_high}")
        
        if rsi_stats['rsi_mean'] < config.rsi_neutral_low or rsi_stats['rsi_mean'] > config.rsi_neutral_high:
            print(f"   ⚠️ La moyenne RSI ({rsi_stats['rsi_mean']:.1f}) est en dehors de la plage neutre!")
            
        # Suggestions d'amélioration
        print(f"\n💡 SUGGESTIONS D'AMÉLIORATION:")
        if ema_crossover == 0:
            print("   • Aucun croisement EMA détecté - vérifier les périodes EMA")
        if rsi_neutral_count < len(analysis_df) * 0.1:
            print(f"   • Plage RSI trop restrictive - essayer {config.rsi_neutral_low-10}-{config.rsi_neutral_high+10}")
        if supertrend_bull < len(analysis_df) * 0.3:
            print("   • SuperTrend rarement bullish - ajuster les paramètres")
            
    else:
        print("❌ Colonnes d'indicateurs manquantes - impossible d'analyser les conditions")

print(f"\n" + "=" * 35)

🔍 DIAGNOSTIC AVANCÉ DES SIGNAUX
📊 Échantillon analysé: 5,000 lignes

📈 COLONNES D'INDICATEURS:
   • EMA Fast (ema_12): ✅
   • EMA Slow (ema_26): ✅
   • RSI (rsi_14): ✅
   • SuperTrend Dir (supertrend_dir_10_3.0): ✅

📊 ANALYSE DES CONDITIONS (sur 5,000 lignes):
   • EMA Fast > Slow: 2,575 (51.5%)
   • EMA Bullish Cross: 73 (1.5%)
   • RSI Neutral (45-55): 1,392 (27.8%)
   • SuperTrend Bullish: 2,531 (50.6%)
   • 🎯 SIGNAUX D'ACHAT FINAUX: 9

📊 STATISTIQUES RSI:
   • Min: 7.7
   • Q25: 41.4
   • Moyenne: nan
   • Q75: 60.3
   • Max: 95.0
   • Plage neutre configurée: 45-55

💡 SUGGESTIONS D'AMÉLIORATION:

📊 Échantillon analysé: 5,000 lignes

📈 COLONNES D'INDICATEURS:
   • EMA Fast (ema_12): ✅
   • EMA Slow (ema_26): ✅
   • RSI (rsi_14): ✅
   • SuperTrend Dir (supertrend_dir_10_3.0): ✅

📊 ANALYSE DES CONDITIONS (sur 5,000 lignes):
   • EMA Fast > Slow: 2,575 (51.5%)
   • EMA Bullish Cross: 73 (1.5%)
   • RSI Neutral (45-55): 1,392 (27.8%)
   • SuperTrend Bullish: 2,531 (50.6%)
   • 🎯 SIGNAU

In [7]:
# 🔍 DIAGNOSTIC SUPERTREND SPÉCIFIQUE
print("🔍 DIAGNOSTIC SUPERTREND")
print("=" * 25)

supertrend_dir_col = config.get_indicator_columns()["supertrend_dir"]

# Analyser les valeurs SuperTrend de façon simplifiée
unique_values = sample_df.select(pl.col(supertrend_dir_col).unique()).to_series().to_list()
print(f"📊 Valeurs uniques de {supertrend_dir_col}:")
for value in unique_values:
    if value is not None:
        count = sample_df.filter(pl.col(supertrend_dir_col) == value).height
        percentage = (count / len(sample_df)) * 100
        print(f"   • Valeur {value}: {count:,} occurrences ({percentage:.1f}%)")
    else:
        null_count = sample_df.filter(pl.col(supertrend_dir_col).is_null()).height
        print(f"   • Valeur NULL: {null_count:,} occurrences ({null_count/len(sample_df)*100:.1f}%)")

# Vérifier s'il y a des NaN/null
null_count = sample_df.select(pl.col(supertrend_dir_col).is_null().sum()).item()
print(f"   • Valeurs nulles: {null_count:,}")

# Vérifier les dernières valeurs pour tendance récente
recent_supertrend = sample_df.tail(100).select([
    pl.col('datetime'),
    pl.col(supertrend_dir_col)
]).tail(10)

print(f"\n📈 Dernières valeurs SuperTrend:")
for row in recent_supertrend.to_dicts():
    print(f"   • {row['datetime']}: {row[supertrend_dir_col]}")

print("=" * 25)

🔍 DIAGNOSTIC SUPERTREND
📊 Valeurs uniques de supertrend_dir_10_3.0:
   • Valeur -1.0: 2,459 occurrences (49.2%)
   • Valeur 1.0: 2,531 occurrences (50.6%)
   • Valeur nan: 10 occurrences (0.2%)
   • Valeurs nulles: 0

📈 Dernières valeurs SuperTrend:
   • 2019-11-29 12:00:00: 1.0
   • 2019-11-29 16:00:00: 1.0
   • 2019-11-29 20:00:00: 1.0
   • 2019-11-30 00:00:00: 1.0
   • 2019-11-30 04:00:00: 1.0
   • 2019-11-30 08:00:00: 1.0
   • 2019-11-30 12:00:00: 1.0
   • 2019-11-30 16:00:00: 1.0
   • 2019-11-30 20:00:00: 1.0
   • 2019-12-01 00:00:00: 1.0


In [8]:
# 🚀 STRATÉGIE SIMPLIFIÉE SANS SUPERTREND (TEST)
print("🚀 TEST STRATÉGIE SIMPLIFIÉE")
print("=" * 30)

# Modifier temporairement la stratégie pour ignorer SuperTrend
def compute_simple_strategy_signals(df: pl.DataFrame) -> pl.DataFrame:
    """Version simplifiée sans SuperTrend pour test"""
    
    ema_fast_col = config.get_indicator_columns()["ema_fast"]
    ema_slow_col = config.get_indicator_columns()["ema_slow"]
    rsi_col = config.get_indicator_columns()["rsi_14"]
    
    # Calcul des signaux simplifiés (sans SuperTrend)
    signals_df = df.with_columns([
        # === CONDITIONS EMA ===
        (pl.col(ema_fast_col) > pl.col(ema_slow_col)).alias("ema_fast_above_slow"),
        (pl.col(ema_fast_col).shift(1) <= pl.col(ema_slow_col).shift(1)).alias("ema_was_below_or_equal"),
        
        # Crossover EMA avec validation de continuité
        ((pl.col(ema_fast_col) > pl.col(ema_slow_col)) & 
         (pl.col(ema_fast_col).shift(1) <= pl.col(ema_slow_col).shift(1)) &
         (pl.int_range(pl.len()) > 0)).alias("ema_bullish_cross"),
        
        # === CONDITIONS RSI (assouplies) ===
        ((pl.col(rsi_col) >= 30) & (pl.col(rsi_col) <= 70)).alias("rsi_ok"),
        
        # === SIGNAL SIMPLIFIÉ ===
        # Achat : Croisement EMA bullish + RSI OK
        ((pl.col(ema_fast_col) > pl.col(ema_slow_col)) & 
         (pl.col(ema_fast_col).shift(1) <= pl.col(ema_slow_col).shift(1)) &
         (pl.int_range(pl.len()) > 0) &
         (pl.col(rsi_col) >= 30) & (pl.col(rsi_col) <= 70)).alias("buy_signal"),
        
        # Vente : Croisement EMA bearish
        ((pl.col(ema_fast_col) <= pl.col(ema_slow_col)) & 
         (pl.col(ema_fast_col).shift(1) > pl.col(ema_slow_col).shift(1)) &
         (pl.int_range(pl.len()) > 0)).alias("sell_signal")
    ])
    
    return signals_df

# Test sur échantillon
test_simple = compute_simple_strategy_signals(sample_df)

buy_count_simple = test_simple.select(pl.col("buy_signal").sum()).item()
sell_count_simple = test_simple.select(pl.col("sell_signal").sum()).item()
ema_cross_count = test_simple.select(pl.col("ema_bullish_cross").sum()).item()
rsi_ok_count = test_simple.select(pl.col("rsi_ok").sum()).item()

print(f"📊 RÉSULTATS STRATÉGIE SIMPLIFIÉE:")
print(f"   • Croisements EMA bullish: {ema_cross_count:,}")
print(f"   • RSI OK (30-70): {rsi_ok_count:,} ({rsi_ok_count/len(sample_df)*100:.1f}%)")  
print(f"   • 🎯 Signaux d'ACHAT: {buy_count_simple:,}")
print(f"   • 🎯 Signaux de VENTE: {sell_count_simple:,}")

if buy_count_simple > 0:
    print(f"\n✅ SUCCÈS ! La stratégie simplifiée génère des signaux.")
    print(f"💡 Le problème vient bien du SuperTrend qui contient uniquement des NaN")
else:
    print(f"\n⚠️ Même la stratégie simplifiée ne génère pas de signaux.")
    print(f"💡 Vérifier les données EMA et RSI")

print("=" * 30)

🚀 TEST STRATÉGIE SIMPLIFIÉE
📊 RÉSULTATS STRATÉGIE SIMPLIFIÉE:
   • Croisements EMA bullish: 73
   • RSI OK (30-70): 4,194 (83.9%)
   • 🎯 Signaux d'ACHAT: 69
   • 🎯 Signaux de VENTE: 72

✅ SUCCÈS ! La stratégie simplifiée génère des signaux.
💡 Le problème vient bien du SuperTrend qui contient uniquement des NaN


In [15]:
def run_chunked_backtest() -> List[Dict]:
    """Exécute le backtesting par chunks sur toutes les données avec validation de cohérence"""
    
    print("🚀 DÉMARRAGE DU BACKTESTING CHUNKED AVEC VALIDATION")
    print("=" * 60)
    
    # Récupérer le nombre total de lignes
    if not data_summary:
        print("❌ Pas d'informations sur les données")
        return []
    
    total_rows = data_summary['total_rows']
    estimated_chunks = (total_rows // config.chunk_size) + 1
    
    print(f"📊 Total à traiter: {total_rows:,} lignes")
    print(f"🔄 Chunks estimés: {estimated_chunks:,}")
    print(f"🛡️ Buffer de contexte: {backtester.state['context_buffer_size']} lignes")
    print(f"⏱️ Début: {datetime.now().strftime('%H:%M:%S')}")
    print()
    
    all_results = []
    start_time = datetime.now()
    total_warnings = 0
    continuity_issues = 0
    
    # Traitement chunk par chunk
    for offset in range(0, total_rows, config.chunk_size):
        chunk_num = (offset // config.chunk_size) + 1
        current_chunk_size = min(config.chunk_size, total_rows - offset)
        
        print(f"[{chunk_num:>3}/{estimated_chunks}] ", end="")
        print(f"Chunk {offset:,}-{offset + current_chunk_size:,} ", end="")
        
        try:
            # Charger le chunk avec contexte
            chunk_start = datetime.now()
            chunk_df = backtester.load_chunk_with_context(offset, current_chunk_size)
            
            if len(chunk_df) == 0:
                print("⚠️ Chunk vide - arrêt")
                break
            
            # Traiter le chunk
            is_first = (offset == 0)
            chunk_result = backtester.process_chunk(chunk_df, is_first)
            
            # Vérifier les erreurs
            if 'error' in chunk_result:
                print(f"❌ Erreur: {chunk_result['error']}")
                break
            
            # Calculer les métriques du chunk
            chunk_time = (datetime.now() - chunk_start).total_seconds()
            rows_per_sec = chunk_result['rows_processed'] / max(chunk_time, 0.001)
            
            # Affichage des métriques avec validation
            continuity_status = "✅" if chunk_result.get('continuity_validated', False) else "⚠️"
            context_info = f"ctx:{chunk_result.get('context_size', 0)}" if chunk_result.get('context_size', 0) > 0 else "no-ctx"
            
            print(f"| {chunk_result['rows_processed']:>5} lignes ", end="")
            print(f"| 📈 {chunk_result['buy_signals']:>2} achats ", end="")
            print(f"| 📉 {chunk_result['sell_signals']:>2} ventes ", end="")
            print(f"| ⚡ {rows_per_sec:>6.0f} l/s ", end="")
            print(f"| {continuity_status} {context_info} ", end="")
            print(f"| 🔄 {backtester.state['total_trades']:>3} trades")
            
            # Gestion des warnings
            if 'warnings' in chunk_result and chunk_result['warnings']:
                for warning in chunk_result['warnings']:
                    print(f"    ⚠️ {warning}")
                    total_warnings += 1
            
            if not chunk_result.get('continuity_validated', False):
                continuity_issues += 1
            
            all_results.append(chunk_result)
            
        except Exception as e:
            print(f"❌ Erreur: {e}")
            break
    
    # Statistiques finales avec validation
    total_time = (datetime.now() - start_time).total_seconds()
    total_processed = sum(r['rows_processed'] for r in all_results)
    total_buy_signals = sum(r['buy_signals'] for r in all_results)
    total_sell_signals = sum(r['sell_signals'] for r in all_results)
    
    print()
    print("=" * 60)
    print("✅ BACKTESTING CHUNKED TERMINÉ AVEC VALIDATION")
    print("=" * 60)
    print(f"📊 Lignes traitées: {total_processed:,}")
    print(f"📈 Total signaux achat: {total_buy_signals:,}")
    print(f"📉 Total signaux vente: {total_sell_signals:,}")
    print(f"🔄 Total trades complétés: {backtester.state['total_trades']:,}")
    print(f"⏱️ Temps total: {total_time:.1f}s")
    print(f"⚡ Performance: {total_processed/max(total_time, 0.001):,.0f} lignes/sec")
    
    # Rapport de validation
    print(f"\n🛡️ RAPPORT DE VALIDATION:")
    print(f"   • Chunks traités: {len(all_results):,}")
    print(f"   • Problèmes de continuité: {continuity_issues:,}")
    print(f"   • Warnings total: {total_warnings:,}")
    
    if continuity_issues == 0 and total_warnings == 0:
        print("   ✅ Cohérence parfaite - tous les signaux sont fiables")
    elif continuity_issues > 0:
        print(f"   ⚠️ {continuity_issues} chunks avec problèmes de continuité")
    
    return all_results

# Exécution du backtesting avec validation
backtest_results = run_chunked_backtest()

🚀 DÉMARRAGE DU BACKTESTING CHUNKED AVEC VALIDATION
📊 Total à traiter: 5,844 lignes
🔄 Chunks estimés: 1
🛡️ Buffer de contexte: 100 lignes
⏱️ Début: 00:40:37

[  1/1] Chunk 0-5,844 |  5844 lignes | 📈 10 achats | 📉 66 ventes | ⚡  13145 l/s | ✅ no-ctx | 🔄   1 trades

✅ BACKTESTING CHUNKED TERMINÉ AVEC VALIDATION
📊 Lignes traitées: 5,844
📈 Total signaux achat: 10
📉 Total signaux vente: 66
🔄 Total trades complétés: 1
⏱️ Temps total: 0.4s
⚡ Performance: 13,140 lignes/sec

🛡️ RAPPORT DE VALIDATION:
   • Chunks traités: 1
   • Problèmes de continuité: 0
   • Warnings total: 0
   ✅ Cohérence parfaite - tous les signaux sont fiables
|  5844 lignes | 📈 10 achats | 📉 66 ventes | ⚡  13145 l/s | ✅ no-ctx | 🔄   1 trades

✅ BACKTESTING CHUNKED TERMINÉ AVEC VALIDATION
📊 Lignes traitées: 5,844
📈 Total signaux achat: 10
📉 Total signaux vente: 66
🔄 Total trades complétés: 1
⏱️ Temps total: 0.4s
⚡ Performance: 13,140 lignes/sec

🛡️ RAPPORT DE VALIDATION:
   • Chunks traités: 1
   • Problèmes de continuité: 

## 5. 💾 Sauvegarde des Résultats dans Table Test

In [16]:
def save_results_to_test_table(results: List[Dict]) -> bool:
    """Sauvegarde les résultats dans la table test pour analyse VectorBT"""
    
    if not results:
        print("❌ Pas de résultats à sauvegarder")
        return False
    
    print("💾 SAUVEGARDE DES RÉSULTATS")
    print("=" * 30)
    
    try:
        # Combiner tous les DataFrames de résultats
        all_data = []
        for result in results:
            if 'data' in result and result['data'] is not None:
                # Ajouter les métadonnées du chunk
                chunk_data = result['data'].with_columns([
                    pl.lit(result['chunk_id']).alias('chunk_id'),
                    pl.lit(datetime.now().isoformat()).alias('backtest_timestamp'),
                    pl.lit(config.symbol).alias('symbol'),
                    pl.lit(f"{config.strategy_name if hasattr(config, 'strategy_name') else 'smart_momentum'}").alias('strategy_name')
                ])
                all_data.append(chunk_data)
        
        if not all_data:
            print("❌ Pas de données à combiner")
            return False
        
        # Combiner toutes les données
        final_df = pl.concat(all_data)
        
        print(f"📊 Données combinées: {len(final_df):,} lignes")
        print(f"📅 Période: {final_df['datetime'].min()} → {final_df['datetime'].max()}")
        
        # Générer le chemin de sortie avec timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = f"{config.test_table_path}backtest_{config.symbol}_{timestamp}.parquet"
        
        # Sauvegarder localement en attendant la correction MinIO
        local_output_path = f"/tmp/backtest_{config.symbol}_{timestamp}.parquet"
        
        # Sauvegarder en local d'abord
        final_df.write_parquet(local_output_path, compression='snappy')
        print(f"✅ Sauvegarde locale: {local_output_path}")
        
        # Essayer MinIO en option
        try:
            temp_table = "temp_backtest_results"
            data_loader.con.register(temp_table, final_df.to_arrow())
            
            export_query = f"""
                COPY (SELECT * FROM {temp_table})
                TO '{output_path}'
                (FORMAT PARQUET, COMPRESSION 'snappy')
            """
            data_loader.con.execute(export_query)
            print(f"✅ Sauvegarde MinIO: {output_path}")
        except Exception as minio_error:
            print(f"⚠️ MinIO non disponible: {minio_error}")
            print(f"📁 Utilisation sauvegarde locale: {local_output_path}")
            output_path = local_output_path
        
        print(f"✅ Résultats sauvegardés: {output_path}")
        print(f"📁 Taille: ~{final_df.estimated_size('mb'):.1f} MB")
        
        # Sauvegarder également les métadonnées
        metadata = {
            'config': {
                'symbol': config.symbol,
                'chunk_size': config.chunk_size,
                'start_date': config.start_date,
                'end_date': config.end_date,
                'initial_cash': config.initial_cash,
                'fees': config.fees
            },
            'results': {
                'total_rows': len(final_df),
                'total_chunks': len(results),
                'total_buy_signals': sum(r['buy_signals'] for r in results),
                'total_sell_signals': sum(r['sell_signals'] for r in results),
                'total_trades': backtester.state['total_trades']
            },
            'paths': {
                'data_path': output_path,
                'source_path': config.feature_store_path
            },
            'timestamp': timestamp
        }
        
        metadata_path = f"{config.test_table_path}metadata_{config.symbol}_{timestamp}.json"
        
        # Sauvegarder les métadonnées (méthode simplifiée)
        metadata_df = pl.DataFrame([metadata])
        data_loader.con.register("temp_metadata", metadata_df.to_arrow())
        data_loader.con.execute(f"""
            COPY (SELECT * FROM temp_metadata)
            TO '{metadata_path.replace('.json', '.parquet')}'
            (FORMAT PARQUET)
        """)
        
        print(f"📋 Métadonnées sauvegardées: {metadata_path.replace('.json', '.parquet')}")
        
        return True
        
    except Exception as e:
        print(f"❌ Erreur lors de la sauvegarde: {e}")
        return False

# Sauvegarder les résultats
save_success = save_results_to_test_table(backtest_results)

💾 SAUVEGARDE DES RÉSULTATS
📊 Données combinées: 5,844 lignes
📅 Période: 2023-01-01 00:00:00 → 2025-08-31 20:00:00
✅ Sauvegarde locale: /tmp/backtest_BTCUSDT_20251004_004046.parquet
⚠️ MinIO non disponible: Invalid Error: Unexpected response while initializing S3 multipart upload
📁 Utilisation sauvegarde locale: /tmp/backtest_BTCUSDT_20251004_004046.parquet
✅ Résultats sauvegardés: /tmp/backtest_BTCUSDT_20251004_004046.parquet
📁 Taille: ~1.6 MB
❌ Erreur lors de la sauvegarde: Invalid Error: Unexpected response while initializing S3 multipart upload


## 6. 📈 Validation et Analyse avec VectorBT

In [18]:
def analyze_with_vectorbt() -> Optional[vbt.Portfolio]:
    """Analyse des résultats avec VectorBT"""
    
    if not backtest_results:
        print("❌ Pas de résultats à analyser")
        return None
    
    print("📈 ANALYSE VECTORBT")
    print("=" * 20)
    
    try:
        # Combiner toutes les données pour VectorBT
        all_data = []
        for result in backtest_results:
            if 'data' in result and result['data'] is not None:
                all_data.append(result['data'])
        
        if not all_data:
            print("❌ Pas de données à analyser")
            return None
        
        # Combiner et convertir en pandas pour VectorBT
        combined_df = pl.concat(all_data)
        df_pd = combined_df.to_pandas().set_index('datetime')
        
        print(f"📊 Données pour VectorBT: {len(df_pd):,} lignes")
        print(f"📅 Période: {df_pd.index.min()} → {df_pd.index.max()}")
        
        # Créer le portfolio VectorBT
        portfolio = vbt.Portfolio.from_signals(
            close=df_pd['close'],
            entries=df_pd['buy_signal'],
            exits=df_pd['sell_signal'],
            init_cash=config.initial_cash,
            fees=config.fees,
            freq='4H'  # Ajuster selon vos données
        )
        
        # Statistiques de base
        print(f"\n📊 RÉSULTATS VECTORBT:")
        print(f"💰 Capital initial: ${config.initial_cash:,.2f}")
        print(f"💰 Capital final: ${portfolio.final_value():,.2f}")
        print(f"📈 Rendement total: {(portfolio.final_value() / config.initial_cash - 1) * 100:.2f}%")
        print(f"🔄 Nombre de trades: {portfolio.trades.count()}")
        
        if portfolio.trades.count() > 0:
            print(f"💹 Trade moyen: {portfolio.trades.pnl.mean():.2f}")
            print(f"🎯 Taux de réussite: {portfolio.trades.win_rate() * 100:.1f}%")
            try:
                print(f"📉 Drawdown max: {portfolio.drawdown().max() * 100:.2f}%")
            except:
                print(f"📉 Drawdown max: N/A")
        
        # Statistiques avancées
        stats = portfolio.stats()
        print(f"\n📊 STATISTIQUES AVANCÉES:")
        print(stats)
        
        return portfolio
        
    except Exception as e:
        print(f"❌ Erreur lors de l'analyse VectorBT: {e}")
        return None

# Analyse avec VectorBT
portfolio = analyze_with_vectorbt()

📈 ANALYSE VECTORBT
📊 Données pour VectorBT: 5,844 lignes
📅 Période: 2023-01-01 00:00:00 → 2025-08-31 20:00:00

📊 RÉSULTATS VECTORBT:
💰 Capital initial: $10,000.00
💰 Capital final: $13,705.34
📈 Rendement total: 37.05%
🔄 Nombre de trades: 10
💹 Trade moyen: 370.53
🎯 Taux de réussite: 40.0%
📉 Drawdown max: 0.00%
📉 Drawdown max: 0.00%

📊 STATISTIQUES AVANCÉES:
Start                         2023-01-01 00:00:00
End                           2025-08-31 20:00:00
Period                          974 days 00:00:00
Start Value                               10000.0
End Value                            13705.338196
Total Return [%]                        37.053382
Benchmark Return [%]                   554.727443
Max Gross Exposure [%]                      100.0
Total Fees Paid                        236.514826
Max Drawdown [%]                        11.481144
Max Drawdown Duration           319 days 16:00:00
Total Trades                                   10
Total Closed Trades                       

In [12]:
# Visualisation des résultats
if portfolio is not None:
    print("🎨 Génération des graphiques...")
    
    # Graphique principal du portfolio
    fig = portfolio.plot()
    fig.show()
    
    # Graphique des trades
    if portfolio.trades.count() > 0:
        trades_fig = portfolio.trades.plot()
        trades_fig.show()
    
    print("✅ Graphiques générés avec succès")
else:
    print("❌ Impossible de générer les graphiques")

❌ Impossible de générer les graphiques


## 7. 📋 Résumé et Prochaines Étapes

### ✅ Ce qui a été accompli
- Backtesting chunked avec continuité des indicateurs
- Traitement de gros volumes avec mémoire constante
- Sauvegarde des résultats dans tables test
- Analyse et validation avec VectorBT

### 🚀 Prochaines étapes suggérées
1. **Optimisation des paramètres** : Utiliser les résultats pour ajuster la stratégie
2. **Backtesting multi-timeframes** : Tester sur différentes périodes
3. **Stratégies avancées** : Intégrer de nouvelles conditions
4. **Production** : Déployer la stratégie validée

---

In [19]:
# Résumé final
print("" * 60)
print("🎯 RÉSUMÉ DU BACKTESTING CHUNKED")
print("" * 60)

if backtest_results:
    total_processed = sum(r['rows_processed'] for r in backtest_results)
    total_chunks = len(backtest_results)
    total_signals = sum(r['buy_signals'] + r['sell_signals'] for r in backtest_results)
    
    print(f"📊 Données traitées: {total_processed:,} lignes en {total_chunks} chunks")
    print(f"📈 Signaux générés: {total_signals:,}")
    print(f"🔄 Trades complétés: {backtester.state['total_trades']:,}")
    
    if portfolio is not None:
        print(f"💰 Performance: {(portfolio.final_value() / config.initial_cash - 1) * 100:.2f}%")
        print(f"📊 Sharpe Ratio: {portfolio.sharpe_ratio():.2f}" if hasattr(portfolio, 'sharpe_ratio') else "")
    
    print(f"💾 Résultats sauvegardés: {'✅' if save_success else '❌'}")
    print(f"📈 Analyse VectorBT: {'✅' if portfolio is not None else '❌'}")
    
else:
    print("❌ Aucun résultat généré")

print("\n✅ Backtesting terminé avec succès !")
print("🚀 Prêt pour l'optimisation et la production")


🎯 RÉSUMÉ DU BACKTESTING CHUNKED

📊 Données traitées: 5,844 lignes en 1 chunks
📈 Signaux générés: 76
🔄 Trades complétés: 1
💰 Performance: 37.05%
📊 Sharpe Ratio: 1.02
💾 Résultats sauvegardés: ❌
📈 Analyse VectorBT: ✅

✅ Backtesting terminé avec succès !
🚀 Prêt pour l'optimisation et la production
